In [ ]:
import pandas as pd 
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import DutchStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import tiktoken
import json
from openai import OpenAI
import time
import psutil
import tracemalloc
from bertopic import BERTopic
import nbformat
import plotly.express as px
import plotly.graph_objects as go
import umap as UMAP


In [ ]:
###Topic_info is extracted output of BERTopic model runs
r1 = pd.read_csv('topic_infoR13.csv')
r2 = pd.read_csv('topic_infoR2.csv')
r3 = pd.read_csv('topic_infoR3.csv')
r4 = pd.read_csv('topic_infoR4.csv')
r5 = pd.read_csv('topic_infoR5.csv')

In [ ]:
r1 = r1[['Representative_Docs']]
r2 = r2[['Representative_Docs']]
r3 = r3[['Representative_Docs']]
r4 = r4[['Representative_Docs']]
r5 = r5[['Representative_Docs']]

In [ ]:
client = OpenAI(api_key='xxxxxxxxxxxxx')


In [ ]:

# Topic modeling function using GPT-3.5-turbo
def topic_modeling(example):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
      {
        "role": "system",
        "content": "You are an expert in topic modeling customer review data. I am going to provide you with a dataset of customer reviews. Please analyze this data and return a detailed mapping of the most recurring topics. For each topic, list the most associated keywords that represent it. Ensure that the topics and keywords are clear, distinct, and capture the essence of the reviews." 
      },
      {
        "role": "user",
        "content": f"{example}"
      }
    ],
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  try:
    return response.choices[0].message.content
  except (KeyError, IndexError) as e:
    return "Error: Unable to process the response."



In [ ]:
#####This process was run iteritavely for each retailer
# Get the current process
process = psutil.Process()

    # Measure initial CPU times and memory usage
cpu_times_before = process.cpu_times()
memory_usage_before = process.memory_info().rss

    # Start measuring peak memory usage
tracemalloc.start()

    # Measure start time
start_time = time.perf_counter()

    # Apply topic modeling to each chunk
r5['topics'] = r5['Representative_Docs'].apply(topic_modeling)

    # Measure end time
end_time = time.perf_counter()

    # Measure final memory usage
memory_usage_after = process.memory_info().rss

    # Stop measuring peak memory usage and get the result
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

    # Measure final CPU times
cpu_times_after = process.cpu_times()

    # Calculate execution time
execution_time = end_time - start_time

    # Calculate CPU usage
user_time = cpu_times_after.user - cpu_times_before.user
system_time = cpu_times_after.system - cpu_times_before.system
cpu_usage = ((user_time + system_time) / execution_time) * 100 / psutil.cpu_count()

    # Calculate memory usage difference
memory_usage_difference = (memory_usage_after - memory_usage_before) / (1024 ** 2)  # Convert bytes to MB

    # Calculate peak memory usage
memory_usage_peak = peak / (1024 ** 2)  # Convert bytes to MB

    # Print the results
print(f"Execution time for r: {execution_time} seconds")
print(f"CPU usage during execution for r: {cpu_usage}%")
print(f"Peak memory usage during execution for r: {memory_usage_peak} MB")
print(f"Memory usage difference during execution for r: {memory_usage_difference} MB")

#########################
r5[['topics']].to_html('R5gptbert.html')

